In [8]:
import pandas as pd
import numpy as np
from  sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

In [3]:
train = pd.read_csv('train.csv')
item_features = pd.read_csv('item-features.csv')
test = pd.read_csv('test.csv')
user_features = pd.read_csv('user-features.csv')
sub = pd.read_csv("sample-submission.csv")

In [7]:
train

,user_id,item_id,like,timestamp
0,140,342,0,1490936622
1,378,172,1,1490936628
2,150,182,0,1490936650
3,455,17,0,1490936704
4,350,409,0,1490936735
...,...,...,...,...
8669,161,312,0,1491215519
8670,406,208,0,1491215543
8671,196,43,0,1491215576
8672,84,100,0,1491215579


In [9]:
user_features

,user_id,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,30,31
0,0,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297
1,1,0.001204,-0.002725,-0.002546,0.003612,-0.000862,0.001187,0.001404,0.001154,-0.005251,...,-0.007268,-0.001209,0.001942,-0.001870,-0.003451,-0.003451,0.000732,-0.002023,-0.002023,0.000515
2,2,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210
3,3,0.000777,-0.001759,-0.001643,0.002332,-0.000557,0.000766,0.000906,0.000745,-0.003389,...,-0.004691,-0.000781,0.001254,-0.001207,-0.002228,-0.002228,0.000472,-0.001306,-0.001306,0.000332
4,4,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,492,0.000983,-0.002225,-0.002079,0.002949,-0.000704,0.000969,0.001147,0.000942,-0.004287,...,-0.005934,-0.000988,0.001586,-0.001527,-0.002818,-0.002818,0.000597,-0.001652,-0.001652,0.000421
493,493,0.001300,-0.002943,-0.002750,0.003901,-0.000931,0.001282,0.001517,0.001246,-0.005671,...,-0.007850,-0.001306,0.002098,-0.002020,-0.003728,-0.003728,0.000790,-0.002185,-0.002185,0.000556
494,494,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210
495,495,0.000983,-0.002225,-0.002079,0.002949,-0.000704,0.000969,0.001147,0.000942,-0.004287,...,-0.005934,-0.000988,0.001586,-0.001527,-0.002818,-0.002818,0.000597,-0.001652,-0.001652,0.000421


In [85]:
a = user_features[user_features['user_id'] == 140].values
a

array([[ 1.40000000e+02,  1.30026082e-03, -2.94295141e-03,
        -2.74986913e-03,  3.90136485e-03, -9.31214571e-04,
         1.28168775e-03,  1.51681192e-03,  1.24601524e-03,
        -5.67125769e-03, -5.67125769e-03,  6.39555030e-03,
         5.42508170e-04,  4.28403531e-03,  2.44115506e-03,
         1.44405606e-03, -3.92822137e-03, -3.34645885e-04,
        -3.13643584e-03, -1.66204423e-03, -1.66204423e-03,
        -1.38946474e-03, -5.95179978e-03, -7.85030748e-03,
        -1.30639797e-03,  2.09799790e-03, -2.01953403e-03,
        -3.72798518e-03, -3.72798518e-03,  7.90334113e-04,
        -2.18536729e-03, -2.18536729e-03,  5.56280990e-04]])

In [86]:
b = item_features[item_features['item_id'] == 342].values
b

array([[ 3.42000000e+02,  3.47509323e-04, -7.86536849e-04,
        -7.34933439e-04,  1.04268362e-03, -2.48877563e-04,
         3.42545461e-04,  4.05385038e-04,  3.33011579e-04,
        -1.51570737e-03, -1.51570737e-03,  1.70928272e-03,
         1.44991407e-04,  1.14495660e-03,  6.52426134e-04,
         3.85940215e-04, -1.04986132e-03, -8.94378749e-05,
        -8.38247738e-04, -4.44200006e-04, -4.44200006e-04,
        -3.71350073e-04, -1.59068540e-03, -2.09808293e-03,
        -3.49149545e-04,  5.60713523e-04, -5.39743172e-04,
        -9.96345948e-04, -9.96345948e-04,  2.11225677e-04,
        -5.84063976e-04, -5.84063976e-04,  1.48672348e-04]])

In [90]:
c = np.multiply(a, b)
c[0]

array([4.78800000e+04, 4.51852759e-07, 2.31473973e-06, 2.02097078e-06,
       4.06788922e-06, 2.31758413e-07, 4.39036322e-07, 6.14892859e-07,
       4.14937501e-07, 8.59596711e-06, 8.59596711e-06, 1.09318036e-05,
       7.86590230e-08, 4.90503450e-06, 1.59267336e-06, 5.57319305e-07,
       4.12408768e-06, 2.99300168e-08, 2.62911025e-06, 7.38280059e-07,
       7.38280059e-07, 5.15977835e-07, 9.46744101e-06, 1.64705961e-05,
       4.56128258e-07, 1.17637579e-06, 1.09002970e-06, 3.71436292e-06,
       3.71436292e-06, 1.66938858e-07, 1.27639431e-06, 1.27639431e-06,
       8.27036011e-08])

In [6]:
b = item_features[item_features['item_id'] == 150].values
a = user_features[user_features['user_id'] == 0].values
c = np.concatenate((a[0], b[0]))
c

array([ 0.00000000e+00,  6.95018646e-04, -1.57307370e-03, -1.46986688e-03,
        2.08536723e-03, -4.97755125e-04,  6.85090922e-04,  8.10770076e-04,
        6.66023158e-04, -3.03141475e-03, -3.03141475e-03,  3.41856543e-03,
        2.89982814e-04,  2.28991320e-03,  1.30485227e-03,  7.71880430e-04,
       -2.09972264e-03, -1.78875750e-04, -1.67649548e-03, -8.88400012e-04,
       -8.88400012e-04, -7.42700147e-04, -3.18137080e-03, -4.19616585e-03,
       -6.98299090e-04,  1.12142705e-03, -1.07948634e-03, -1.99269190e-03,
       -1.99269190e-03,  4.22451353e-04, -1.16812795e-03, -1.16812795e-03,
        2.97344697e-04,  1.50000000e+02,  1.15255803e-03, -2.60864761e-03,
       -2.43749846e-03,  3.45819033e-03, -8.25433494e-04,  1.13609477e-03,
        1.34451007e-03,  1.10447446e-03, -5.02703265e-03, -5.02703265e-03,
        5.66904943e-03,  4.80882096e-04,  3.79739144e-03,  2.16385269e-03,
        1.28001888e-03, -3.48199609e-03, -2.96631873e-04, -2.78015323e-03,
       -1.47324475e-03, -

In [30]:
X = []
y = []
for row in tqdm(train.values):
    b = item_features[item_features['item_id'] == row[1]].values[0, 1:]
    a = user_features[user_features['user_id'] == row[0]].values[0, 1:]
    c = np.concatenate((a, b))
    X.append(c)
    y.append(row[2])

100%|█████████████████████████████████████████████████████████████████████████████| 8674/8674 [00:14<00:00, 595.88it/s]


In [31]:
X

[array([ 1.30026082e-03, -2.94295141e-03, -2.74986913e-03,  3.90136485e-03,
        -9.31214571e-04,  1.28168775e-03,  1.51681192e-03,  1.24601524e-03,
        -5.67125769e-03, -5.67125769e-03,  6.39555030e-03,  5.42508170e-04,
         4.28403531e-03,  2.44115506e-03,  1.44405606e-03, -3.92822137e-03,
        -3.34645885e-04, -3.13643584e-03, -1.66204423e-03, -1.66204423e-03,
        -1.38946474e-03, -5.95179978e-03, -7.85030748e-03, -1.30639797e-03,
         2.09799790e-03, -2.01953403e-03, -3.72798518e-03, -3.72798518e-03,
         7.90334113e-04, -2.18536729e-03, -2.18536729e-03,  5.56280990e-04,
         3.47509323e-04, -7.86536849e-04, -7.34933439e-04,  1.04268362e-03,
        -2.48877563e-04,  3.42545461e-04,  4.05385038e-04,  3.33011579e-04,
        -1.51570737e-03, -1.51570737e-03,  1.70928272e-03,  1.44991407e-04,
         1.14495660e-03,  6.52426134e-04,  3.85940215e-04, -1.04986132e-03,
        -8.94378749e-05, -8.38247738e-04, -4.44200006e-04, -4.44200006e-04,
        -3.7

In [10]:
df = pd.DataFrame(X)
df['like'] = y
df

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,like
0,140.0,0.001300,-0.002943,-0.002750,0.003901,-0.000931,0.001282,0.001517,0.001246,-0.005671,...,-0.000349,0.000561,-0.000540,-0.000996,-0.000996,0.000211,-0.000584,-0.000584,0.000149,0
1,378.0,0.003204,-0.007252,-0.006776,0.009613,-0.002295,0.003158,0.003737,0.003070,-0.013974,...,-0.002263,0.003634,-0.003498,-0.006457,-0.006457,0.001369,-0.003785,-0.003785,0.000964,1
2,150.0,0.002554,-0.005780,-0.005401,0.007662,-0.001829,0.002517,0.002979,0.002447,-0.011138,...,-0.001710,0.002747,-0.002644,-0.004881,-0.004881,0.001035,-0.002861,-0.002861,0.000728,0
3,455.0,0.004649,-0.010523,-0.009833,0.013950,-0.003330,0.004583,0.005424,0.004455,-0.020279,...,-0.002208,0.003546,-0.003414,-0.006301,-0.006301,0.001336,-0.003694,-0.003694,0.000940,0
4,350.0,0.000919,-0.002081,-0.001944,0.002759,-0.000658,0.000906,0.001073,0.000881,-0.004010,...,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8669,161.0,0.000602,-0.001362,-0.001273,0.001806,-0.000431,0.000593,0.000702,0.000577,-0.002625,...,-0.000349,0.000561,-0.000540,-0.000996,-0.000996,0.000211,-0.000584,-0.000584,0.000149,0
8670,406.0,0.002357,-0.005335,-0.004985,0.007072,-0.001688,0.002323,0.002749,0.002259,-0.010280,...,-0.000605,0.000971,-0.000935,-0.001726,-0.001726,0.000366,-0.001012,-0.001012,0.000258,0
8671,196.0,0.001667,-0.003772,-0.003525,0.005001,-0.001194,0.001643,0.001944,0.001597,-0.007269,...,-0.001397,0.002243,-0.002159,-0.003985,-0.003985,0.000845,-0.002336,-0.002336,0.000595,0
8672,84.0,0.001433,-0.003243,-0.003030,0.004299,-0.001026,0.001412,0.001671,0.001373,-0.006249,...,-0.001209,0.001942,-0.001870,-0.003451,-0.003451,0.000732,-0.002023,-0.002023,0.000515,0


In [67]:
# X = []
# y = []
# for user in train.user_id:
#     for item in train.item_id:
#         b = item_features[item_features['item_id'] == item].values
#         a = user_features[user_features['user_id'] == user].values
#         c = np.multiply(a, b)
#         X.append(c)

# X

In [75]:
# X = []
# for i in range(train.shape[0]):
#     print(train[train['user_id'] == i])

In [3]:
from sklearn.decomposition import PCA

In [4]:
pca = PCA(32)

In [ ]:
pca.fit()

In [184]:
X_train = df.iloc[:, :-1]
y_train = df.iloc[:, -1:]

In [12]:
from sklearn.linear_model import LogisticRegression

In [27]:
lg = LogisticRegression()

In [32]:
lg.fit(X, y)

LogisticRegression()

In [17]:
X_test = user_features[user_features['user_id'] == 0].values[0, 1:]
X_test

array([ 0.00069502, -0.00157307, -0.00146987,  0.00208537, -0.00049776,
        0.00068509,  0.00081077,  0.00066602, -0.00303141, -0.00303141,
        0.00341857,  0.00028998,  0.00228991,  0.00130485,  0.00077188,
       -0.00209972, -0.00017888, -0.0016765 , -0.0008884 , -0.0008884 ,
       -0.0007427 , -0.00318137, -0.00419617, -0.0006983 ,  0.00112143,
       -0.00107949, -0.00199269, -0.00199269,  0.00042245, -0.00116813,
       -0.00116813,  0.00029734])

In [22]:
X_ar = []
for j in range(item_features.shape[0]):
    b = item_features.loc[j, :].values[1:]
    a = X_test
    c = np.concatenate((a, b))
    X_ar.append(c)

In [23]:
X_ar

[array([ 0.00069502, -0.00157307, -0.00146987,  0.00208537, -0.00049776,
         0.00068509,  0.00081077,  0.00066602, -0.00303141, -0.00303141,
         0.00341857,  0.00028998,  0.00228991,  0.00130485,  0.00077188,
        -0.00209972, -0.00017888, -0.0016765 , -0.0008884 , -0.0008884 ,
        -0.0007427 , -0.00318137, -0.00419617, -0.0006983 ,  0.00112143,
        -0.00107949, -0.00199269, -0.00199269,  0.00042245, -0.00116813,
        -0.00116813,  0.00029734,  0.00564636, -0.01277971, -0.01194125,
         0.0169416 , -0.00404378,  0.0055657 ,  0.00658673,  0.0054108 ,
        -0.02462733, -0.02462733,  0.02777256,  0.00235583,  0.01860334,
         0.01060067,  0.00627079, -0.01705823, -0.00145319, -0.01361991,
        -0.0072174 , -0.0072174 , -0.00603372, -0.02584558, -0.03408981,
        -0.00567301,  0.00911052, -0.00876979, -0.01618871, -0.01618871,
         0.00343201, -0.00948992, -0.00948992,  0.00241564]),
 array([ 0.00069502, -0.00157307, -0.00146987,  0.00208537, -0

In [34]:
prd = lg.predict_proba(X_ar)

In [35]:
prd[:, 1]

array([0.40335989, 0.19498717, 0.15101351, 0.21293093, 0.11375622,
       0.19744374, 0.38184414, 0.11375622, 0.10557853, 0.12922713,
       0.0158263 , 0.73382201, 0.35876123, 0.35881141, 0.14439414,
       0.10557853, 0.16317214, 0.09364886, 0.13532314, 0.13133161,
       0.1617136 , 0.0297975 , 0.16317214, 0.08761436, 0.26221416,
       0.13532314, 0.08654477, 0.13336017, 0.18233029, 0.09129613,
       0.11013991, 0.12230095, 0.16886715, 0.11375622, 0.72631379,
       0.17436985, 0.13722873, 0.07616447, 0.16023961, 0.64619874,
       0.2516984 , 0.11375622, 0.13722873, 0.10557853, 0.11013991,
       0.16604605, 0.10557853, 0.11013991, 0.12473278, 0.10557853,
       0.12473278, 0.23108946, 0.22098106, 0.13336017, 0.19375038,
       0.23440521, 0.10557853, 0.10557853, 0.11970292, 0.12922713,
       0.12230095, 0.10557853, 0.13532314, 0.10557853, 0.13532314,
       0.11375622, 0.15571541, 0.13133161, 0.10557853, 0.14940003,
       0.12922713, 0.02813332, 0.10557853, 0.10557853, 0.11013

In [205]:
t = []
for row in test.values:
    d = user_features[user_features['user_id'] == row[0]].values
    t.append(d[0])

In [212]:
X_test = pd.DataFrame(t)
X_test.drop([0], axis = 1)

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,0.002382,-0.005392,-0.005038,0.007148,-0.001706,0.002348,0.002779,0.002283,-0.010391,-0.010391,...,-0.014384,-0.002394,0.003844,-0.003700,-0.006831,-0.006831,0.001448,-0.004004,-0.004004,0.001019
1,0.000491,-0.001112,-0.001039,0.001475,-0.000352,0.000484,0.000573,0.000471,-0.002144,-0.002144,...,-0.002967,-0.000494,0.000793,-0.000763,-0.001409,-0.001409,0.000299,-0.000826,-0.000826,0.000210
2,0.000695,-0.001573,-0.001470,0.002085,-0.000498,0.000685,0.000811,0.000666,-0.003031,-0.003031,...,-0.004196,-0.000698,0.001121,-0.001079,-0.001993,-0.001993,0.000422,-0.001168,-0.001168,0.000297
3,0.000777,-0.001759,-0.001643,0.002332,-0.000557,0.000766,0.000906,0.000745,-0.003389,-0.003389,...,-0.004691,-0.000781,0.001254,-0.001207,-0.002228,-0.002228,0.000472,-0.001306,-0.001306,0.000332
4,0.000777,-0.001759,-0.001643,0.002332,-0.000557,0.000766,0.000906,0.000745,-0.003389,-0.003389,...,-0.004691,-0.000781,0.001254,-0.001207,-0.002228,-0.002228,0.000472,-0.001306,-0.001306,0.000332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,0.004963,-0.011234,-0.010497,0.014893,-0.003555,0.004893,0.005790,0.004756,-0.021649,-0.021649,...,-0.029967,-0.004987,0.008009,-0.007709,-0.014231,-0.014231,0.003017,-0.008342,-0.008342,0.002123
493,0.001253,-0.002836,-0.002650,0.003759,-0.000897,0.001235,0.001462,0.001201,-0.005465,-0.005465,...,-0.007565,-0.001259,0.002022,-0.001946,-0.003592,-0.003592,0.000762,-0.002106,-0.002106,0.000536
494,0.005060,-0.011452,-0.010701,0.015182,-0.003624,0.004988,0.005902,0.004849,-0.022069,-0.022069,...,-0.030549,-0.005084,0.008164,-0.007859,-0.014507,-0.014507,0.003075,-0.008504,-0.008504,0.002165
495,0.001099,-0.002487,-0.002324,0.003297,-0.000787,0.001083,0.001282,0.001053,-0.004793,-0.004793,...,-0.006635,-0.001104,0.001773,-0.001707,-0.003151,-0.003151,0.000668,-0.001847,-0.001847,0.000470


In [114]:
# X_test = test['user_id'].values
# X_test

In [142]:
# X_test = test.iloc[:, :1]
# X_test=X_test['user_id']


In [215]:
pred = lg.predict(X_test)

In [216]:
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Еще 1 вариант

In [219]:
for user in train[train.like == 1]['user_id'].values:
    a = user_features[user_features['user_id'] == row[0]].values
    print(a)

[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.911

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.911

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

...

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4.82760969e-03 -8.91158907e-03
  -8.91158907e-03  1.88925989e-03 -5.22402701e-03 -5.22402701e-03
   1.32976591e-03]]
[[ 1.67000000e+02  3.10821787e-03 -7.03499945e-03 -6.57344452e-03
   9.32604578e-03 -2.22602859e-03  3.06381974e-03  3.62587401e-03
   2.97854611e-03 -1.35568989e-02 -1.35568989e-02  1.52882894e-02
   1.29684257e-03  1.02408031e-02  5.83547674e-03  3.45195422e-03
  -9.39024513e-03 -7.99956672e-04 -7.49751570e-03 -3.97304564e-03
  -3.97304564e-03 -3.32145603e-03 -1.42275227e-02 -1.87658242e-02
  -3.12288847e-03  5.01517422e-03 -4

In [ ]:
X = []
y = []
for row in train.values:
    b = item_features[item_features['item_id'] == row[1]].values
    a = user_features[user_features['user_id'] == row[0]].values
    c = np.multiply(a, b)
    X.append(a)
    y.append(row[2])